In [24]:
import pandas as pd
import folium
from folium import Map, Marker

## Take a sample trajectory and convert to HTML

In [25]:
sample_trj_id = "62155"
df = pd.read_feather(f'data/{sample_trj_id}.ftr').sort_values(by='pingtimestamp')
#df = df[df['distance'] != 0]
df

,trj_id,driving_mode,osname,pingtimestamp,rawlat,rawlng,speed,bearing,accuracy,time,day_of_week,month,year
397,62155,car,ios,1555652978,-6.152760,106.817673,8.696515,167,8.0,13:49:38,4,4,2019
328,62155,car,ios,1555652979,-6.152830,106.817696,8.006205,167,4.0,13:49:39,4,4,2019
304,62155,car,ios,1555652980,-6.152903,106.817711,8.006205,167,6.0,13:49:40,4,4,2019
104,62155,car,ios,1555652981,-6.152982,106.817726,8.249675,167,6.0,13:49:41,4,4,2019
642,62155,car,ios,1555652982,-6.153068,106.817749,9.676609,167,4.0,13:49:42,4,4,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,62155,car,ios,1555654026,-6.165999,106.800819,4.879760,92,12.0,14:07:06,4,4,2019
418,62155,car,ios,1555654028,-6.165983,106.800926,5.049112,93,16.0,14:07:08,4,4,2019
486,62155,car,ios,1555654029,-6.165990,106.801079,5.597365,93,16.0,14:07:09,4,4,2019
238,62155,car,ios,1555654030,-6.165983,106.801140,5.597365,93,24.0,14:07:10,4,4,2019


In [26]:
df.describe(include='all')

,trj_id,driving_mode,osname,pingtimestamp,rawlat,rawlng,speed,bearing,accuracy,time,day_of_week,month,year
count,645,645,645,6.450000e+02,645.000000,645.000000,645.000000,645.000000,645.000000,645,645.0,645.0,645.0
unique,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,645,NaN,NaN,NaN
top,62155,car,ios,NaN,NaN,NaN,NaN,NaN,NaN,13:49:38,NaN,NaN,NaN
freq,645,645,645,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
mean,NaN,NaN,NaN,1.555654e+09,-6.163095,106.809105,8.774761,220.291473,6.508527,NaN,4.0,4.0,2019.0
std,NaN,NaN,NaN,3.439817e+02,0.003555,0.009152,4.107210,72.316089,2.047269,NaN,0.0,0.0,0.0
min,NaN,NaN,NaN,1.555653e+09,-6.166734,106.791298,0.155046,31.000000,3.000000,NaN,4.0,4.0,2019.0
25%,NaN,NaN,NaN,1.555653e+09,-6.166095,106.800285,6.324945,166.000000,6.000000,NaN,4.0,4.0,2019.0
50%,NaN,NaN,NaN,1.555654e+09,-6.165110,106.812294,8.683396,262.000000,6.000000,NaN,4.0,4.0,2019.0
75%,NaN,NaN,NaN,1.555654e+09,-6.160484,106.817955,11.946131,266.000000,8.000000,NaN,4.0,4.0,2019.0


In [27]:
#df.loc[0, 'location']
[df.quantile(0.5)['rawlat'], df.quantile(0.5)['rawlng']]

[-6.165109634399414, 106.81229400634766]

Polyline over Marker due to the number of pings = number of points to be connected.

Marker is more laggy if there are too many of them in the same HTML.

In [28]:
sgp_map = folium.Map(
    location=[df.quantile(0.5)['rawlat'], df.quantile(0.5)['rawlng']],
    tiles="openstreetmap",
    zoom_start=17
    )

"""
for idx, row in df.iterrows():
    Marker([row['lat'], row['lon']], icon=folium.Icon(color="red")).add_to(sgp_map)

for idx, row in df.iterrows():
    Marker([row['location'][1], row['location'][0]], icon=folium.Icon(color="green")).add_to(sgp_map)
"""

# Creating feature groups
f1 = folium.FeatureGroup("Red")
f2 = folium.FeatureGroup("Green")

# Adding lines to the different feature groups
folium.vector_layers.PolyLine(list(zip(df['rawlat'], df['rawlng'])), popup='<b>Original Path</b>', color='red', weight=5).add_to(f1)
#folium.vector_layers.PolyLine(list(map(lambda x: x[::-1], df['location'])), popup='<b>Valhalla Path</b>', color='green',weight=5).add_to(f2)

f1.add_to(sgp_map)
#f2.add_to(sgp_map)
folium.LayerControl().add_to(sgp_map)

In [29]:
sgp_map.save(f'data/valhalla/line/{sample_trj_id}.html')